<a href="https://colab.research.google.com/github/rishurnjan/Multi-label_classification/blob/main/BERT_for_multilabel_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv("/content/goodreads_data.csv",index_col=0,on_bad_lines='skip')
df=df.drop_duplicates()
df.dropna(inplace=True)
sample_df = df.sample(n=1500, random_state=42)

sample_df['combined']=sample_df['Book']+" . "+sample_df['Author']+" . "+sample_df['Description']
import re

# Clean and extract unique genres
unique_genres = set(
    re.sub(r"[\[\]']", '', genre).strip()  # Remove brackets and quotes
    for genres in sample_df['Genres'].apply(lambda x: x.split(','))
    for genre in genres
    if genre.strip() not in ['', '[', ']']
)
unique_genres.remove('')
print(unique_genres)



{'School', 'Halloween', 'Southern Gothic', 'Mystery', 'Dutch Literature', '18th Century', 'Natural History', 'Czech Literature', 'Science', 'Brain', 'Iran', 'Arthurian', 'Classical Music', 'Physics', 'Fiction', 'History', 'Fantasy', 'Swedish Literature', 'Light Novel', 'Fairies', 'Historical Fiction', 'Witches', 'Presidents', 'Crime', 'How To', 'Anti Racist', 'Kids', 'Linguistics', 'Superheroes', 'Feminism', 'Race', 'Environment', 'Love Story', 'Teen', 'Relationships', 'Sweden', '16th Century', 'Pop Culture', 'Adult Fiction', 'Humor', 'Southern', 'Murder Mystery', 'Sports', 'Popular Science', 'Turkish Literature', 'Astronomy', 'Folklore', 'Society', 'Mermaids', 'World War II', 'Novels', 'Drama', 'Genderfluid', 'Language', 'Transgender', 'Entrepreneurship', 'Design', 'Cyberpunk', 'Banned Books', 'Personal Development', 'Menage', 'Hinduism', 'Germany', 'Journalism', 'Theology', 'Literature', 'Social Justice', 'Christmas', 'Psychological Thriller', 'Lesbian Romance', 'Japanese Literature'

In [2]:
len(unique_genres)

397

In [3]:
for genre in unique_genres:
    sample_df[genre] = sample_df['Genres'].apply(lambda x: 1 if genre in x.split(',') else 0)
sample_df=sample_df.drop(columns=['Book','Author','Description','Genres','Avg_Rating','Num_Ratings','URL'])
sample_df.head()

<ipython-input-3-127421bb4283>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sample_df[genre] = sample_df['Genres'].apply(lambda x: 1 if genre in x.split(',') else 0)
<ipython-input-3-127421bb4283>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sample_df[genre] = sample_df['Genres'].apply(lambda x: 1 if genre in x.split(',') else 0)
<ipython-input-3-127421bb4283>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

,combined,School,Halloween,Southern Gothic,Mystery,Dutch Literature,18th Century,Natural History,Czech Literature,Science,...,Medieval,Architecture,Buisness,Urbanism,Historical Fantasy,Ghosts,Self Help,Leadership,Terrorism,Regency
5291,Groundwork of the Metaphysics of Morals . Imma...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
510,The Silmarillion . J.R.R. Tolkien . The Silmar...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4347,Cancer Ward . Aleksandr Solzhenitsyn . One of ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8424,"The Tyrant’s Tomb (The Trials of Apollo, #4) ....",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9088,Dix raisons (possibles) à la tristesse de pens...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(sample_df, test_size=0.2, random_state=42)
test_df, val_df=train_test_split(test_df, test_size=0.5, random_state=42)

In [5]:
# Hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-05
THRESHOLD = 0.5 # threshold for the sigmoid

In [6]:
import torch
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        self.title = list(df['combined'])
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'title': title
        }


In [7]:
target_list=list(sample_df.columns)
target_list.remove('combined')
len(target_list)
#

397

In [8]:
import torch, transformers
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, len(target_list))

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
save_path = '/content/MLTC_model_state.bin'
torch.save(model.state_dict(), save_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN, target_list)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN, target_list)
test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN, target_list)

In [10]:
# Data loaders
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [11]:
import numpy as np
import tqdm as tq

loss_fn = nn.BCEWithLogitsLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def train_model(training_loader, model, optimizer):

    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to training mode (activate droput, batch norm)
    model.train()
    model.to(device)
    # initialize the progress bar
    loop = tq.tqdm(enumerate(training_loader), total=len(training_loader),
                      leave=True, colour='steelblue')
    for batch_idx, data in loop:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        # forward
        outputs = model(ids, mask, token_type_ids) # (batch,predict)=(32,8)
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())
        # training accuracy, apply sigmoid, round (apply thresh 0.5)
        outputs = torch.sigmoid(outputs).cpu().detach().numpy().round()
        targets = targets.cpu().detach().numpy()
        correct_predictions += np.sum(outputs==targets)
        num_samples += targets.size   # total number of elements in the 2D array

        # backward
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # grad descent step
        optimizer.step()

        # Update progress bar
        #loop.set_description(f"")
        #loop.set_postfix(batch_loss=loss)

    # returning: trained model, model accuracy, mean loss
    return model, float(correct_predictions)/num_samples, np.mean(losses)

In [12]:
import random

param_grid = {
    'learning_rate': [1e-5, 2e-5, 3e-5],
    'batch_size': [8, 16, 32],
    'epochs': [2, 3, 4]
}

def random_search(train_loader, model, param_grid, device, num_iter=5):
    best_model = None
    best_accuracy = 0.0
    best_params = None

    for i in range(num_iter):
        params = {k: random.choice(v) for k, v in param_grid.items()}
        print(f"Running with params: {params}")

        optimizer = torch.optim.AdamW(model.parameters(), lr=params['learning_rate'])

        for epoch in range(params['epochs']):
            trained_model, train_acc, train_loss = train_model(train_loader, model, optimizer)
            print(f"Epoch {epoch+1}/{params['epochs']}, Train Accuracy: {train_acc}, Train Loss: {train_loss}")

        val_acc = train_acc

        if val_acc > best_accuracy:
            best_accuracy = val_acc
            best_model = trained_model.state_dict()
            best_params = params

    print(f"Best model found with params: {best_params}, Accuracy: {best_accuracy}")
    return best_model, best_params


In [13]:
best_model_state, best_params = random_search(train_data_loader, model, param_grid, device)
torch.save(best_model_state, '/content/best_model.bin')


Running with params: {'learning_rate': 3e-05, 'batch_size': 32, 'epochs': 2}


/usr/local/lib/python3.10/dist-packages/tqdm/std.py:161: TqdmWarning: Unknown colour (steelblue); valid choices: [hex (#00ff00), BLACK, RED, GREEN, YELLOW, BLUE, MAGENTA, CYAN, WHITE]
  self.colour = colour
100%|██████████| 38/38 [00:46<00:00,  1.23s/it]


Epoch 1/2, Train Accuracy: 0.7767611251049539, Train Loss: 0.5420691888583334


100%|██████████| 38/38 [00:47<00:00,  1.25s/it]


Epoch 2/2, Train Accuracy: 0.9806696053736356, Train Loss: 0.29087405063604055
Running with params: {'learning_rate': 1e-05, 'batch_size': 8, 'epochs': 2}


...

Epoch 4/4, Train Accuracy: 1.0, Train Loss: 0.004753556604938288
Running with params: {'learning_rate': 1e-05, 'batch_size': 8, 'epochs': 2}


100%|██████████| 38/38 [00:49<00:00,  1.30s/it]


Epoch 1/2, Train Accuracy: 1.0, Train Loss: 0.003533382924567712


100%|██████████| 38/38 [00:49<00:00,  1.30s/it]


Epoch 2/2, Train Accuracy: 1.0, Train Loss: 0.0026961610603489375
Best model found with params: {'learning_rate': 2e-05, 'batch_size': 16, 'epochs': 3}, Accuracy: 1.0


In [14]:
def get_predictions(model, data_loader):

    model = model.eval()  # Set the model to evaluation mode

    titles = []
    predictions = []
    prediction_probs = []
    target_values = []

    with torch.no_grad():  # Disable gradient calculations for inference
        for data in data_loader:
            title = data["title"]
            ids = data["input_ids"].to(device, dtype=torch.long)
            mask = data["attention_mask"].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data["targets"].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)

            # Apply sigmoid for probabilities
            outputs = torch.sigmoid(outputs).detach().cpu()

            # Threshold at 0.5 for binary classification
            preds = outputs.round()

            # Store results
            titles.extend(title)
            predictions.append(preds)
            prediction_probs.append(outputs)
            target_values.append(targets.cpu())

    # Concatenate the results into tensors
    predictions = torch.cat(predictions, dim=0)
    prediction_probs = torch.cat(prediction_probs, dim=0)
    target_values = torch.cat(target_values, dim=0)

    return titles, predictions, prediction_probs, target_values


In [15]:
titles, predictions, prediction_probs, true_labels = get_predictions(model, test_data_loader)

# Now you can evaluate the predictions using sklearn metrics
from sklearn.metrics import classification_report

print(classification_report(true_labels, predictions, target_names=target_list))


                              precision    recall  f1-score   support

                      School       0.00      0.00      0.00         0
                   Halloween       0.00      0.00      0.00         0
             Southern Gothic       0.00      0.00      0.00         0
                     Mystery       0.00      0.00      0.00         0
            Dutch Literature       0.00      0.00      0.00         0
                18th Century       0.00      0.00      0.00         0
             Natural History       0.00      0.00      0.00         0
            Czech Literature       0.00      0.00      0.00         0
                     Science       0.00      0.00      0.00         0
                       Brain       0.00      0.00      0.00         0
                        Iran       0.00      0.00      0.00         0
                   Arthurian       0.00      0.00      0.00         0
             Classical Music       0.00      0.00      0.00         0
                   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471:

In the real system below feedback data would be user provided.

In [16]:
def collect_feedback(titles, predictions, true_labels):
    feedback_data = []
    for title, pred, true in zip(titles, predictions, true_labels):
        if not torch.equal(pred, true):
            feedback_data.append({
                'title': title,
                'correct_label': true.tolist()
            })

    return feedback_data



def create_feedback_loader(feedback_data, tokenizer, max_len, target_list, batch_size=8):

    feedback_df = pd.DataFrame(feedback_data)
    for idx, genre in enumerate(target_list):
        feedback_df[genre] = feedback_df['correct_label'].apply(lambda x: x[idx])

    feedback_df = feedback_df.drop(columns=['correct_label'])
    feedback_df['combined'] = feedback_df['title']

    feedback_dataset = CustomDataset(feedback_df, tokenizer, max_len, target_list)
    feedback_loader = torch.utils.data.DataLoader(
        feedback_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )

    return feedback_loader


def fine_tune_model_on_feedback(model, feedback_loader, optimizer, epochs=2):

    model.train()
    for epoch in range(epochs):
        model, train_acc, train_loss = train_model(feedback_loader, model, optimizer)
        print(f"Fine-tuning Epoch {epoch+1}/{epochs} - Loss: {train_loss}, Accuracy: {train_acc}")

    return model

def continuous_learning(model, data_loader, tokenizer, target_list, max_len, optimizer, epochs=2):
    titles, predictions, _, true_labels = get_predictions(model, data_loader)

    feedback_data = collect_feedback(titles, predictions, true_labels)

    if len(feedback_data) == 0:
        print("No feedback available (model is performing perfectly).")
        return model

    feedback_loader = create_feedback_loader(feedback_data, tokenizer, max_len, target_list)

    model = fine_tune_model_on_feedback(model, feedback_loader, optimizer, epochs)

    return model

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

fine_tuned_model = continuous_learning(
    model,
    test_data_loader,
    tokenizer,
    target_list,
    MAX_LEN,
    optimizer,
    epochs=2
)


No feedback available (model is performing perfectly).
